<a href="https://colab.research.google.com/github/EdwardDixon/trainings_in_ml/blob/master/Designing_(simple!)_Experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Designing (simple!) Experiments

How much data is enough? Was it a fluke?  For experimental results to be meaningful, we need to consider the role that chance may have played in our results.

We'll study the problem by doing the classic experiment - flipping a simulated coin, to see how many times it comes up heads.  A fair coin would turn up "heads" on 50% of flips - but is this coin fair?  Let's leave the next code cell collapsed to keep a sense of mystery.

In [0]:
#@title
from numpy.random import random 

def flip_dodgy_coin(weight = 0.65):
    if random() > weight:
        return 1
    
    return 0

## How fair is my coin?
The collapsed section above conceals a coin of "unknown fairness" (we don't know how often it comes up heads).  Let's start with a few tosses to observe its behavior...

In [0]:
def experimental_flips(number_of_flips):
    results = {'heads':0, 'tails':0}

    for i in range(0, number_of_flips):
        if flip_dodgy_coin() == 0:
            results['heads'] += 1
        else:
            results['tails'] += 1

    return(results)


print(experimental_flips(10))

What would the picture look like if we repeated our flips a lot more times?

In [0]:
%matplotlib inline
from matplotlib import pyplot as plt
import pandas as pd

def run_a_lot_of_experiments(how_many_batches = 1000, flips_per_batch=10):
    results = []
    
    for i in range(0, how_many_batches):
        batch_results = experimental_flips(flips_per_batch)
        probability_heads = batch_results['heads'] / flips_per_batch
        results.append(probability_heads)
        
    return (results)

df_experiments = pd.DataFrame({'experiments':run_a_lot_of_experiments(1000, 10)})
df_experiments.hist()

We can see the there are a wide range of outcomes for this experiment - sometimes we are getting 9 heads from 10 flips, someone only 1 or 2.  Suspiciouly, the "peak" is not over 0.5!

Since we are working with a proportion, the binomial theorem describes the distribution of our coin tosses.  We need to figure how variable our results will be with the number of samples we have. There is a simple formula for the "standard error" ($\sigma_e$) of the sample:

$$\sigma_e = \sqrt{\frac{p(1-p)}{n}}$$

- $p$ is the ratio of heads to total coin tosses that we observed
- $n$ is the total number of tosses

If we make $n$ bigger, we can see $\sigma_e$ is going to get smaller.  We get big gains at first, but the $\sqrt{}$ warns us the gains taper off...

In [0]:
from math import sqrt

p = 0.5
n_list = range(10,10000)

def calculate_standard_error(p, n):
    result = sqrt((p * (1-p)) / n)
    return (result)

sigma_e = []
for n in n_list:
    sigma_e.append(calculate_standard_error(p, n))
    
df = pd.DataFrame({'$\sigma_e$' : sigma_e}, index=n_list)
df.plot.line()

I think we probably all knew that n would be important, but the value of $p$ matters too:

In [0]:
p_list = []
p_one_minus_p = []
for i in range(0,100):
    p = i/100
    p_list.append(p)
    p_one_minus_p.append(p * (1-p))
    
df_p_important = pd.DataFrame({'$p*(1-p)$':p_one_minus_p}, index=p_list)
df_p_important.plot.line()

If we were working with a high value for $p$ - perhaps model accuracy? - we could actually get small error bars from a surprisingly small test set.  If the proportion is more towards the middle, we need more data to narrow the range of plausible values.

So what is the range of likely values for our experiment?  We just need to multiply our standard error by a Z score - how sure do we want to be?

![Z](https://upload.wikimedia.org/wikipedia/commons/thumb/2/25/The_Normal_Distribution.svg/640px-The_Normal_Distribution.svg.png)

If we take Z = 1.96, we get our 95% confidence interval:

In [0]:
p = 0.5
n = 100
small_experiment = experimental_flips(n)
p = small_experiment['heads'] / n

standard_error = calculate_standard_error(p, n)
error_margin = 1.65 * standard_error
print('p(Heads) = %2.2f'%p + ' +- %2.2f'%error_margin + ' with 95% confidence')

What if we wanted a specific confidence interval?  We already know our margin for error, $m$ is given by:

$$m = Z\sigma_e = Z\sqrt{\frac{p(1-p)}{n}} $$

If we assume that we _know_ the margin we want, we can just re-arrange to find the number of samples that will give the 95% confidence interval a width of $2m$:

$$n = 2\frac{p(1-p)}{\frac{m^2}{z^2}}$$

In [0]:
z = 1.96
m = 0.01
samples_required = 2*(p * (1-p)) / (m*m/z*z)
print(samples_required)

By returning to the experiment code, we can try our new value for sample size, and look at the histogram to see if our experiment works out like we calculated it should.